# Question 3 (Solved as optimization problem)

# Optimization Problem:
Let $ R \in \mathbb{R}^{120 * 5}$ be the given returns of the 5 assets over the last 120 months (10 years) where $R_{i,j}, i \in \{0, ..., 119\}, j \in \{0, 2, ..4\}$ is the percent return of the j-th asset in the i-th month.

Now, let $Q := \frac{R}{100} + \mathbb{1}^{120 * 5}$ where $\mathbb{1}^{120 * 5}$ is a matrix of ones with size (120, 5). (i.e., Q is the matrix of monthly returns for each asset such that if $R_{i,j} = 6.02 $ for some i,j representing a 6% return then $Q_{i, j} = 1.0602$).

Let our variable weights be represented by $ w \in [0, 1]^{5 * 1} $

Finally, for simplicity, let the function $MUE(x)$ return the product of all elements in the given vector, x, and let $VAR(x)$ return the sample variance of all elements in the given vector x.

BASIC DERIVATIONS:
Then, using these definitions, the monthly returns for a rebalancing portfolio with given weights $w$ are given by:
\begin{align}
Qw
\end{align}
where $(Qw)_i$  is the monthly return of the portfolio for month i,  $i \in 0, ..., 119$. Conversely, the total return of the portfolio is given by:
\begin{align}
MUE(Qw)
\end{align}
the annualized rate of return is given by:
\begin{align}
(MUE(Qw))^\frac{12}{120} - 1
\end{align}
and the variance of the portfolio is given by:
\begin{align}
VAR(Qw)
\end{align}

CONSTRAINTS:

Our first constraint is that the weights of the portfolio must sum to 1:
\begin{align}
w^T\mathbf{1} = 1
\end{align}
The second constraint is that the weights of the portfolio must be positive (because the portfolio is long-only):
\begin{align}
  w \geq 0
\end{align}
Our third constraint is that the annualized standard deviation must be less than 10% = 0.1. So, since the formula for annualized standard deviation is $12^{0.5}$ * sample standard deviation, our constraint must be:
\begin{align}
test_1 \\
12^\frac{1}{2} \sigma &< 0.1 \\
12 * VAR(Qw) &< 0.01 \\
1200 * VAR(Qw) &< 1 \\
1200 * VAR(Qw) - 1 &< 0 \\
1 - 1200 * VAR(Qw) &> 0 \\
1 - \epsilon - 1200 * VAR(Qw) &\geq 0 \\
\end{align}
where $\epsilon > 0$ is a very small value.

OBJECTIVE:

Our goal is to maximize the annualized rate of return of this portfolio. However, let

\begin{align}
  G(x) := (x + 1)^\frac{120}{12}
\end{align}
and let F(x) be such that:
\begin{align}
  F(x) = x^\frac{12}{120} - 1
\end{align}
where $F(MUE(Qw)) = $ annualized rate of return. This means that $G(F(x)) = x$ and thus $G(F(MUE(Qw))) = MUE(Qw)$ where G is a monotonic increasing function by definition. So we have shown that maximizing $F(MUE(Qw))$ is equivalent to maximizing $MUE(Qw)$ itself.


PROBLEM FORMULATION:

So, combining our constraints and objective, we arrive at the optimization problem:
\begin{align}
  max_{w \in [0,1]^{5*1}} \text{  }&MUE(Qw) \\
  s.t. \text{ } & w^T\mathbf{1} = 1 \\
  & w \geq 0 \\
   1 - \epsilon - 1200 * & VAR(Qw) \geq 0 \\
\end{align}










In [22]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import math
df1 = pd.read_excel("IA_Project_Data.xlsx", sheet_name = 0)
asset_array = (df1.drop(columns = ["Unnamed: 0"]).dropna().to_numpy()) /100 + 1
Q = asset_array[-120:, :]

def ann_var_cons(x):
  # derivation:
  # original constraint: sqrt(12) * SD <= 0.1
  # so:
  # 12 * var <= 0.01
  # 1200 * var <= 1
  # 1 - 1200 * var >=0
  # error term = 0.0001 because using inequality (>=. 0) constraint
  return 1 - 0.0001 -  1200 * np.var(Q @ x)

def ret(x):
  return -np.prod(Q @ x)
def total_weight_constraint(x):
    return np.sum(x)-1.0
def long_only_constraint(x):
    return x

w0 = np.ones(5) / 5
cons = ({'type': 'ineq', 'fun': ann_var_cons},# makes annualized variance <= 0.1
        {'type': 'eq', 'fun': total_weight_constraint},# makes x.T @ 1 == 1
        {'type': 'ineq', 'fun': long_only_constraint})  # makes x >= 0
w = minimize(ret, w0, method='SLSQP', constraints=cons)['x']
total_ret = minimize(ret, w0, method='SLSQP', constraints=cons)['fun']
print("Annualized Std Dev = " + str(math.sqrt(12) * np.std(Q @ w)))
print("Annualized Return Rate = " + str((-total_ret) ** (12/120) - 1))
print("Optimal weights: ")
i = 0
for column in df1.drop(columns = ['Unnamed: 0']).columns:
  print(column + " weight = " + str(w[i]))
  i+= 1


Annualized Std Dev = 0.09999500031083043
Annualized Return Rate = 0.09168821779822323
Optimal weights: 
S&P 500 weight = 0.6908011963966395
MSCI EAFE weight = 1.8323220002912366e-15
US AGGREGATE BOND weight = 8.474734043868726e-17
CITI NON US GOVT BOND weight = 4.014631238124372e-16
US 30 Day T-Bill weight = 0.3091988036033583


# Question 6 (Revised)
Revised my answer to this question by fixing how I calculated SP500 sector returns. Sector returns are now calculated by:

$w_{i,j}$ = weight of i-th company within j-th sector = i-th market cap / sector market cap

$r_i$ = return of i-th company = (10/31 price - 9/30 price) / 9/30 price

j-th sector return =$ \sum_{i=1}^N w_{i,j} * r_i$



In [19]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import math
df2 = pd.read_excel("IA_Project_Data.xlsx", sheet_name = 1)

# Putnam returns
# value before = sum(shares * price on 9/30)
# value after = sum(shares * price on 10/31)
# return = (value after - value before) / value before
df2_clone = df2.copy()
percent_alloc_df = pd.DataFrame()
percent_alloc_df['Sectors'] = df2_clone.Sector.unique()
percent_alloc_df = percent_alloc_df.set_index('Sectors')

df2_clone['Putnam 09-30'] = df2_clone['Shares in Putnam Equity Fund'].multiply(df2_clone['Price On 09/30/2022'])
df2_clone['Putnam 10-31'] = df2_clone['Shares in Putnam Equity Fund'].multiply(df2_clone['Price On 10/31/2022'])
putnam_09_30_val = df2_clone['Putnam 09-30'].sum()
putnam_10_31_val = df2_clone['Putnam 10-31'].sum()
putnam_ret = (putnam_10_31_val - putnam_09_30_val) / putnam_09_30_val
print("a) Putnam return = " + str(putnam_ret))

# SP500 returns

#total_SP_mkt_cap = df2_clone['Market Cap ($ Millions)'].multiply(df2_clone['S&P 500 Inclusion']).sum()
#df2_clone['Percentage of S&P 500'] = (df2_clone['Market Cap ($ Millions)'].multiply(df2_clone['S&P 500 Inclusion'])) / total_SP_mkt_cap
total_SP_mkt_cap = (df2_clone['Market Cap ($ Millions)'][df2_clone['S&P 500 Inclusion'] == True]).sum()
df2_clone['Percentage of S&P 500'] = (df2_clone['Market Cap ($ Millions)'][df2_clone['S&P 500 Inclusion'] == True]) / total_SP_mkt_cap
SP500_09_30_val = df2_clone['Percentage of S&P 500'].multiply(df2_clone['Price On 09/30/2022']).sum()
SP500_10_31_val = df2_clone['Percentage of S&P 500'].multiply(df2_clone['Price On 10/31/2022']).sum()
SP500_ret = (SP500_10_31_val - SP500_09_30_val) / SP500_09_30_val

# active return is difference between Putnam return and benchmark



# d)
# for SP500:
total_ret = 0
SP500_sector_rets = np.zeros(len(df2_clone.Sector.unique()))
i = 0
sector_sums = 0
df2_clone['company rets'] = (df2_clone['Price On 10/31/2022'] - df2_clone['Price On 09/30/2022']) / df2_clone['Price On 09/30/2022']
for sector in df2_clone.Sector.unique():
  ### REVISED THIS SECTION
  total_sector_mkt_cap = df2_clone['Market Cap ($ Millions)'][(df2_clone['Sector'] == sector) & (df2_clone['S&P 500 Inclusion'] == True)].sum()
  df2_clone['Percentage of S&P500 ' + str(sector)] = df2_clone['Market Cap ($ Millions)'][(df2_clone['Sector'] == sector) & (df2_clone['S&P 500 Inclusion'] == True)] / total_sector_mkt_cap
  sector_return = (df2_clone['Percentage of S&P500 ' + str(sector)] * df2_clone['company rets']).sum()
  #print(sector + " return in SP = " + str(sector_return))
  total_ret += sector_return * (total_sector_mkt_cap / total_SP_mkt_cap)
  SP500_sector_rets[i] = sector_return
  sector_sums += total_sector_mkt_cap
  i += 1
#print("SP sector mkt cap sum = " + str(sector_sums))
#print("SP total mkt cap = " + str(total_SP_mkt_cap))
print("b) Total ret SP = " + str(total_ret))
SP500_total_ret_sum = total_ret
print("c) Active Return = " + str(total_ret - putnam_ret))
percent_alloc_df['S&P500 Returns'] = SP500_sector_rets
## SEEMS TO BE SOME DESCREPANCY BETWEEN TOTAL SP500 RETURN AND THE WEIGHTED SUM OF ALL SECTOR RETURNS
### PROBABLY NOT FLOATING POINT ERROR BASED ON SIZE --> CHECK THIS OUT!!
# Putnam returns by sector:
print("d)")
print("e)")
print("f)")
putnam_sector_rets = np.zeros(len(df2_clone.Sector.unique()))
i = 0
total_ret = 0
for sector in df2_clone.Sector.unique():

  putnam_sector_val_09_30 = df2_clone['Shares in Putnam Equity Fund'][df2_clone['Sector'] == sector].multiply(df2_clone['Price On 09/30/2022']).sum()
  putnam_sector_val_10_31 = df2_clone['Shares in Putnam Equity Fund'][df2_clone['Sector'] == sector].multiply(df2_clone['Price On 10/31/2022']).sum()
  if(putnam_sector_val_09_30 > 0):
    sector_return = (putnam_sector_val_10_31 - putnam_sector_val_09_30) / putnam_sector_val_09_30
    putnam_sector_rets[i] = sector_return
    i += 1
    putnam_sector_weight = putnam_sector_val_09_30 / putnam_09_30_val
    total_ret += putnam_sector_weight * sector_return


putnam_total_ret_sum = total_ret
#print("Total putnam ret = " + str(total_ret)) ## this matches yay
percent_alloc_df['Putnam Returns'] = putnam_sector_rets

# e)
# get percent of each sector that SP500 holds

percent_alloc_array = np.zeros((len(df2_clone.Sector.unique()), 2))
i = 0
for sector in df2_clone.Sector.unique():
  total_sector_mkt_cap = (df2_clone['Market Cap ($ Millions)'][(df2_clone['Sector'] == sector) & (df2_clone['S&P 500 Inclusion'] == True)]).sum()
  #print("Percent of S&P in " + sector + " = " + str(total_sector_mkt_cap / total_SP_mkt_cap))
  percent_alloc_array[i , 0] = total_sector_mkt_cap / total_SP_mkt_cap
  i += 1

# get percent of total portfolio that Putnam holds in each sector
i = 0
for sector in df2_clone.Sector.unique():
  putnam_sector_investment = df2_clone['Shares in Putnam Equity Fund'][df2_clone['Sector'] == sector].multiply(df2_clone['Price On 09/30/2022']).sum()
  #print("Percent of Putnam Fund in " + sector + " = " + str(putnam_sector_investment / putnam_09_30_val))
  percent_alloc_array[i , 1] = putnam_sector_investment / putnam_09_30_val
  i += 1
# compare them
percent_alloc_df['S&P 500 weights'] = percent_alloc_array[:,0]
percent_alloc_df['Putnam weights'] = percent_alloc_array[:,1]
percent_alloc_df['Putnam - S&P weight'] = percent_alloc_df['Putnam weights'] - percent_alloc_df['S&P 500 weights']


### f)


### Allocation
B_s = percent_alloc_df['Putnam weights'].multiply(percent_alloc_df['S&P500 Returns']).sum()
B = SP500_total_ret_sum
R = putnam_total_ret_sum
print("active return from sum = " + str(putnam_total_ret_sum - SP500_total_ret_sum))
percent_alloc_df['Allocation'] = (percent_alloc_df['Putnam weights'] - percent_alloc_df['S&P 500 weights']).multiply(percent_alloc_df['S&P500 Returns'])
print("B_s - B = " +str(B_s - B))
print("sum(Allocation) = contribution to arithmetic excess ret from sector alloc. = " + str(percent_alloc_df['Allocation'].sum()))

### Selection
R_s = percent_alloc_df['S&P 500 weights'].multiply(percent_alloc_df['Putnam Returns']).sum()
percent_alloc_df['Selection'] = (percent_alloc_df['Putnam Returns'] - percent_alloc_df['S&P500 Returns']).multiply(percent_alloc_df['S&P 500 weights'])
print("R_s - B = " + str(R_s - B))
print("sum(Selection) = contribution to arithmetic excess return from selection = " +  str(percent_alloc_df['Selection'].sum()))

### Interaction
percent_alloc_df['Interaction'] = (percent_alloc_df['Putnam weights'] - percent_alloc_df['S&P 500 weights']).multiply(
    percent_alloc_df['Putnam Returns'] - percent_alloc_df['S&P500 Returns'])
print("R - R_s - B_s + B = " + str(R - R_s - B_s + B))
print("sum(Interaction) = contribution to arithmetic excess return from interaction = " + str(percent_alloc_df['Interaction'].sum()))
print("sum(Allocation) + sum(Selection) + sum(Interaction) = " + str(percent_alloc_df['Allocation'].sum() + percent_alloc_df['Selection'].sum() + percent_alloc_df['Interaction'].sum()))
print(percent_alloc_df * 100)

a) Putnam return = 0.0731096175158555
b) Total ret SP = 0.07518768608472039
c) Active Return = 0.002078068568864888
d)
e)
f)
active return from sum = -0.002078068568864791
B_s - B = 0.0003607959095644764
sum(Allocation) = contribution to arithmetic excess ret from sector alloc. = 0.0003607959095644712
R_s - B = 0.004254972096010751
sum(Selection) = contribution to arithmetic excess return from selection = 0.004254972096010752
R - R_s - B_s + B = -0.006693836574440018
sum(Interaction) = contribution to arithmetic excess return from interaction = -0.006693836574440028
sum(Allocation) + sum(Selection) + sum(Interaction) = -0.002078068568864805
                        S&P500 Returns  Putnam Returns  S&P 500 weights  \
Sectors                                                                   
Industrials                  13.695985       11.732390         7.461151   
Health Care                   9.588466        9.359077        13.965743   
Information Technology        8.111944        8.131